# Advent of code 2024
## Challenge 21
## Part 1
### https://adventofcode.com/2024/day/21#part1

What I understood at some point is that a different combination of keys with the same length may lead to a different combination of 

In [1]:
from collections import deque
import re

The variables below model the keypads as a maze that is later used with a pathfinding algorithm.

In [2]:
numeric_keypad = [
    ['7', '8', '9'],
    ['4', '5', '6'],
    ['1', '2', '3'],
    ['#', '0', 'A']
    ]

numeric_keypad_to_position = {
    "7": (0,0),
    "8": (0,1),
    "9": (0,2),
    "4": (1,0),
    "5": (1,1),
    "6": (1,2),
    "1": (2,0),
    "2": (2,1),
    "3": (2,2),
    "0": (3,1),
    "A": (3,2)
}

In [3]:
directional_keypad = [
    ['#', '^', 'A'],
    ['<', 'v', '>']
    ]

directional_keypad_to_position = {
    "^": (0,1),
    "A": (0,2),
    "<": (1,0),
    "v": (1,1),
    ">": (1,2)
}

In [4]:
direction_to_symbol = {
    (0,-1): "<",
     (0,1): ">",
     (1,0): "v",
    (-1,0): "^",
}

In [5]:
cache = {}
#This algorithm returns all the shortest path possible. It also uses a cache system to be more effective.
def bfs_pathfinding_all_possibilities(maze, start, end):
    # We first see if the path is in the cache
    if (start,end) in cache:
        return cache[(start,end)]
    # Otherwise we find the path
    else:
        cache[(start,end)] = []
        
        rows = len(maze)
        cols = len(maze[0])

        steps = [(0,-1),(0,1),(1,0),(-1,0)]

        queue = deque([[(start[0],start[1]), 0,""]])

        score_by_position = {(start[0],start[1]): 0}

        while queue:

            current_position_data = queue.popleft()
            current_position = current_position_data[0]
            current_score = current_position_data[1]
            # Current paths are carried along. This would not be possible for a big maze, but it is here
            # because the keypads are small mazes.
            current_path = current_position_data[2]

            r, c = current_position

            if current_position == end:
                # When we reach the end, we add the path to the cache. But we don't stop, we continue
                # after.
                new_path = current_path + 'A'
                cache[(start,end)].append(new_path)
                               
            if 0 <= r < rows and 0 <= c < cols:
                for dr, dc in steps:
                    nr, nc = r + dr, c + dc
                    
                    # Here, what allows paths to continue is that one of the conditions is if the length of the path is either shorter or equal
                    # the current shortest path. Such that if it is longer, the path dies, but if it is shorter, a new shortest value is set
                    # and if it is the same length, we keep going with the path.
                    if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] >= current_score + 1):
                        new_path = current_path + direction_to_symbol[(dr,dc)]
                        queue.append([(nr, nc), current_score + 1, new_path])
                        score_by_position[(nr, nc)] =  current_score + 1
        
        return cache[(start,end)]

In [6]:
second_cache = {}
# In this algorithm, we return the first shortest path
def bfs_pathfinding_first_shortest(maze, start, end):
    if (start,end) in second_cache:
        return second_cache[(start,end)]
    else:
        rows = len(maze)
        cols = len(maze[0])

        steps = [(0,-1),(0,1),(1,0),(-1,0)]

        queue = deque([[(start[0],start[1]), 0,""]])

        score_by_position = {(start[0],start[1]): 0}

        while queue:

            current_position_data = queue.popleft()
            current_position = current_position_data[0]
            current_score = current_position_data[1]
            current_path = current_position_data[2]

            r, c = current_position

            if current_position == end:
                new_path = current_path + 'A'
                second_cache[(start,end)] = new_path
                # So for this algorithm, as soon as we reach the end, we stop
                return second_cache[(start,end)]
                               

            if 0 <= r < rows and 0 <= c < cols:
                for dr, dc in steps:
                    nr, nc = r + dr, c + dc
                    
                    # Here, we let a path through only if it is smaller then the current path.
                    if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] > current_score + 1):
                        new_path = current_path + direction_to_symbol[(dr,dc)]
                        queue.append([(nr, nc), current_score + 1, new_path])
                        score_by_position[(nr, nc)] =  current_score + 1

I was lucky enough to quickly figure out what was the "challenging" part of the challenge. And I managed to figure it out without needing to look it up online.

What happens is that two different combination of keys with the same length will not necessarily lead to combinations of keys in the next key pad that will be of the same length. The combinations <A^A>^^AvvvA, <A^A^>^AvvvA, and <A^A^^>AvvvA each all have the same length, are all combinations with the lowest amount of keys, and are all derived from the numeric combination 029A. However, when we recreate those 3 different combinations with the next directional pad, the resulting combinations may vary in length. Not all of the resulting combinations will have the same length. And so that is the twist.

That is why for each keypad except the last one, we have to look at all of the possible combinations. At the start, I only picked out the first combination that had the lowest amount of keys, and discarding the others. And so if the combination was not the right one, it would lead to a derived combination that was longer then what it should have been.

For the last keypad, it is ok to not look at all the possible combinations, because we will not derive another combination from it. And so finding the first shortest combination is enough. And we preserve only the lowest length.

In [ ]:
#combination_list = ["029A","980A","179A","456A","379A"]
combination_list = ["463A","340A","129A","083A","341A"]
combination = ""
total = 0

for value in combination_list:

    combination = 'A' + value

    numeric_path_list = []

    first_direction_paths = [""]
    temporary_list = []
    second_temporary_list = []

    second_direction_paths = [""]
    second_direction_path_list = []
    second_direction_path_lists = []
    second_direction_paths_final = []

    temporary_string = ""
    third_direction_paths = []
    third_direction_path_list = []
    third_direction_path_lists = []

    # Here, we create a list of tuples representing the paths that need to be walked to press the buttons in the right order
    for index in range(len(combination) - 1):
        numeric_path_list.append((combination[index],combination[index + 1]))
    
    # This part of the algorithm is the crux of the challenge. This is where we create all of the possible derived paths, out
    # of the possible paths required to navigate between keys.
    for numeric_path in numeric_path_list:
        # We extract all of the possible shortest paths between 2 keys.
        temporary_list = bfs_pathfinding_all_possibilities(numeric_keypad,numeric_keypad_to_position[numeric_path[0]],numeric_keypad_to_position[numeric_path[1]])
        # We start with the first paths. So let's say there are 3 ways to navigate between the first keys, and 2 ways
        # to navigate between the next two keys, there will now be 6 resulting possible paths. We start the looping with 
        # a list with only an empty string.
        for path in first_direction_paths:
            # We loop through all the found shortest ways
            for temporary_path in temporary_list:
                # That's when we combine: previous path with new path
                second_temporary_list.append(path + temporary_path)
        # These newly created paths now become the "current" or "previous" paths that will be looped through
        # along with the newly found paths
        first_direction_paths = second_temporary_list.copy()
        # We reset the list.
        second_temporary_list = []

    # We add an A to each path because all directional pads start at the A key
    for index in range(len(first_direction_paths)):
        first_direction_paths[index] = 'A' + first_direction_paths[index]

    # We create the new key combinations to see from which key to which key we have to move
    # we do that for each created paths in the first stage.
    for path in first_direction_paths:
        for index in range(len(path) - 1):
            second_direction_path_list.append((path[index],path[index + 1]))
        second_direction_path_lists.append(second_direction_path_list)
        second_direction_path_list = []

    # This looping is the same as the previous one, but it now has an extra loop to loop through all the paths
    # that have been created in the previous steps
    for path_list in second_direction_path_lists:
        for directional_path in path_list:
            temporary_list = bfs_pathfinding_all_possibilities(directional_keypad,directional_keypad_to_position[directional_path[0]],directional_keypad_to_position[directional_path[1]])
            for path in second_direction_paths:
                for temporary_path in temporary_list:
                    second_temporary_list.append(path + temporary_path)
            second_direction_paths = second_temporary_list.copy()
            second_temporary_list = []
        # Here, we put all the resulting paths in a single list. So that in can be reused as a single list in the next step
        for path in second_direction_paths:
            second_direction_paths_final.append(path)
        second_direction_paths = [""]

    # We extract the shortest resulting combination
    length = len(second_direction_paths_final[0])
    for path in second_direction_paths_final:
        if len(path) < length:
            length = len(path)

    # We filter the resulting list to only keep the paths of the shortest length
    second_direction_paths_final = [path for path in second_direction_paths_final if len(path) == length]

    # We insert the A at the start of every resulting combinations
    for index in range(len(second_direction_paths_final)):
        second_direction_paths_final[index] = 'A' + second_direction_paths_final[index]

    # We create another list of paths for the last iteration
    for path in second_direction_paths_final:
        for index in range(len(path) - 1):
            third_direction_path_list.append((path[index],path[index + 1]))
        third_direction_path_lists.append(third_direction_path_list)
        third_direction_path_list = []

    # That's where things get easier here because we now only need to extract the first shortest combination from the
    # previous paths
    for path_list in third_direction_path_lists:
        for directional_path in path_list:
            temporary_string += bfs_pathfinding_first_shortest(directional_keypad,directional_keypad_to_position[directional_path[0]],directional_keypad_to_position[directional_path[1]])
        third_direction_paths.append(temporary_string)
        temporary_string = ""

    # We extract the shortest combination
    third_length = len(third_direction_paths[0])
    for path in third_direction_paths:
        if len(path) < third_length:
            third_length = len(path)

    # We calculate the complexity of the combination 
    combination = combination[1:-1]
    
    while combination[0] == '0':
        combination = combination[1:]
    
    total += third_length * int(combination)
    
print(total)

## Part 2

For this part, I followed the tutorial at https://www.reddit.com/r/adventofcode/comments/1hjx0x4/2024_day_21_quick_tutorial_to_solve_part_2_in/. It was hard enough to just implement that solution.

### 1: Build The Shortest Path Graph for the keypads

In [8]:
# Variables recreating the keypads as mazes on which a BFS pathfinding algorithm is used to find the path from every position
# to every other positions.
numeric_keypad = [
    ['7', '8', '9'],
    ['4', '5', '6'],
    ['1', '2', '3'],
    ['#', '0', 'A']
    ]

numeric_keypad_to_position = {
    "7": (0,0),
    "8": (0,1),
    "9": (0,2),
    "4": (1,0),
    "5": (1,1),
    "6": (1,2),
    "1": (2,0),
    "2": (2,1),
    "3": (2,2),
    "0": (3,1),
    "A": (3,2)
}

position_to_numeric_keypad = {
    (0,0): "7",
    (0,1): "8",
    (0,2): "9",
    (1,0): "4",
    (1,1): "5",
    (1,2): "6",
    (2,0): "1",
    (2,1): "2",
    (2,2): "3",
    (3,1): "0",
    (3,2): "A"
}

directional_keypad = [
    ['#', '^', 'A'],
    ['<', 'v', '>']
    ]

directional_keypad_to_position = {
    "^": (0,1),
    "A": (0,2),
    "<": (1,0),
    "v": (1,1),
    ">": (1,2)
}

position_to_directional_keypad = {
    (0,1): "^",
    (0,2): "A",
    (1,0): "<",
    (1,1): "v",
    (1,2): ">"
}

direction_to_symbol = {
    (0,-1): "<",
     (0,1): ">",
     (1,0): "v",
    (-1,0): "^",
}

In [9]:
cache_part_two = {}
#This algorithm returns all the shortest path possible. It also uses a cache system to be more effective.
def bfs_pathfinding_all_possibilities_two(maze, start, end, position_to_value):
    # We first see if the path is in the cache
    if (position_to_value[start],position_to_value[end]) in cache_part_two:
        return cache_part_two[(position_to_value[start],position_to_value[end])]
    # Otherwise we find the path
    else:
        cache_part_two[(position_to_value[start],position_to_value[end])] = []
                
        rows = len(maze)
        cols = len(maze[0])

        steps = [(0,-1),(0,1),(1,0),(-1,0)]

        queue = deque([[(start[0],start[1]), 0,""]])

        score_by_position = {(start[0],start[1]): 0}

        while queue:

            current_position_data = queue.popleft()
            current_position = current_position_data[0]
            current_score = current_position_data[1]
            # Current paths are carried along. This would not be possible for a big maze, but it is here
            # because the keypads are small mazes.
            current_path = current_position_data[2]

            r, c = current_position

            if current_position == end:
                # When we reach the end, we add the path to the cache. But we don't stop, we continue
                # after.
                cache_part_two[(position_to_value[start],position_to_value[end])].append(current_path)
                               
            if 0 <= r < rows and 0 <= c < cols:
                for dr, dc in steps:
                    nr, nc = r + dr, c + dc
                    
                    # Here, what allows paths to continue is that one of the conditions is if the length of the path is either shorter or equal
                    # the current shortest path. Such that if it is longer, the path dies, but if it is shorter, a new shortest value is set
                    # and if it is the same length, we keep going with the path.
                    if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] >= current_score + 1):
                        new_path = current_path + direction_to_symbol[(dr,dc)]
                        queue.append([(nr, nc), current_score + 1, new_path])
                        score_by_position[(nr, nc)] =  current_score + 1
        
        return cache_part_two[(position_to_value[start],position_to_value[end])]

In [15]:
list_numeric_keypads = ['0','1','2','3','4','5','6','7','8','9','A']
list_directional_keypads = ['<','>','^','v','A']

# From here we find all the shortest paths from every position to every position. Litteraly all paths
for dir_key in list_numeric_keypads:
    for dir_key_two in list_numeric_keypads:
        bfs_pathfinding_all_possibilities_two(numeric_keypad,numeric_keypad_to_position[dir_key],numeric_keypad_to_position[dir_key_two],position_to_numeric_keypad)

for num_key in list_directional_keypads:
    for num_key_two in list_directional_keypads:
        bfs_pathfinding_all_possibilities_two(directional_keypad,directional_keypad_to_position[num_key],directional_keypad_to_position[num_key_two],position_to_directional_keypad)

### 2: Build the output key sequence

In [ ]:
# This function builds a list of all possible sequences for a specific combination
# It uses the cache that was built above.
# This function is necessary because there can be multiple shortest paths with the same length and so
# this leads to multiple combination.
def build_squence(keys, index, previous_key, current_path, result):
    if index == len(keys):
        result.append(current_path)
        return
    for path in cache_part_two[(previous_key,keys[index])]:
        build_squence(keys, index + 1, keys[index], current_path + path + 'A', result)
        
test = []
build_squence('<A', 0, 'A', "", test)
print(test)

### 3: Find the shortest output sequence

In [ ]:
# This is the function that finds the shortest output sequence
import re

# We use a cache here, the whole concept of the algorith is to cache repetitive opeartions 
# to save computing power.
shortest_sequence_cache = {}
def shortest_sequence(keys, depth):
    # Here, we need a total because the function splits keys by the sequences separated by A keys.
    # It keeps the ending A key.
    total = 0
    # When we reach the last depth of 0, the only thing to be done is to return the length
    # of the sequence. The level above will then only retain the sequence at level 0 with the shortest length
    if depth == 0:
        return len(keys)
    # If there is already a cached value, it is returned. Any repetitive computation is then
    # avoided. 
    if (keys,depth) in shortest_sequence_cache:
        return shortest_sequence_cache[(keys,depth)]
    # First thing first is to split the sequence at the A characters. And iterate through all of them
    for sub_key in re.findall(r'.*?A', keys):
        sequence_list = []
        # For each sequence, we build all the possible combinations of keys to achieve this sequence
        build_squence(sub_key, 0, 'A', "", sequence_list)
        minimum = 0
        # We then iterate through each sequence
        for sequence in sequence_list:
            # We assign the result of the first recurcion immediately. And then we compare
            # the result of every other sequence against the minimum.
            # this part of the function contains the recursion. But for the sake of explanation,
            # we can say that the recursion finds the shortest length of the sequence at each level
            if minimum == 0:
                minimum = shortest_sequence(sequence, depth - 1)
            # So here, we only preserve the sequence that provides the shortest length at the lowest depth
            else:
                score = shortest_sequence(sequence, depth - 1)
                if score < minimum:
                    minimum = score
        # We add this value to the total, and then we proceed to the next sequence in the list
        total += minimum
    # Once all the shortest length for each sequence is found, we have the value that we need to add to the cache
    # This takes place at every level, except level 0 (where we just look at the length)
    # So this total ultimately becomes the total for the initial keys and the initial depth provided to the function
    shortest_sequence_cache[(keys,depth)] = total
    return total 

# These tests are the examples provided in the challenge.
print(shortest_sequence('029A', 3) == len('<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A'))
print(shortest_sequence('980A', 3) == len('<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A'))
print(shortest_sequence('179A', 3) == len('<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'))
print(shortest_sequence('456A', 3) == len('<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A'))
print(shortest_sequence('379A', 3) == len('<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'))

In [ ]:
# putting everything together:
# input
combination_list = ["463A","340A","129A","083A","341A"]
total_complexity = 0

# Go through each combination
for combination in combination_list:
    
    # find the shortest length for a depth of 26.
    length = shortest_sequence(combination,26)
    
    # Extract the numbers of each combination
    complexity_factor = combination[:-1]
    
    while complexity_factor[0] == '0':
        complexity_factor = complexity_factor[1:]
    
    # Multiply the complexity factor with the shortest length, and add it to the total
    total_complexity += length * int(complexity_factor)
    
print(total_complexity)